# Diagrams

In [ ]:
# |default_exp diagrams

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from stringdale.core import  checkLogs
import pytest

In [ ]:
#| export
from copy import deepcopy,copy
import itertools
import logging
import asyncio

from collections import defaultdict,OrderedDict
from contextlib import ExitStack
from pprint import pprint
from pydantic import BaseModel

from typing import Optional,Union,List,Dict,Any,Literal,Callable
from fastcore.basics import patch
import logging

from stringdale.core import wrap_exception
from stringdale.schema import (
    DiagramSchema,draw_diagram,BaseModelExtra,DiagramType
    ,_is_attr_method,get_state_key,set_state_key)

## Diagram base class

In [ ]:
#| export
class Diagram():
    """
    An instance of a stringdale diagram. Instantiated by calling the Schema()

    Has the following public attributes:
    output - the output of the last run
    finished - whether the diagram has reached the End node
    state - the current state of the diagram

    """

    def __init__(self,graph,funcs,type,state:BaseModel,schema:DiagramSchema,anon=False,root=None):
        self.graph = graph
        self.funcs = funcs
        self.type = type
        self.state = state
        self.schema = schema
        self.anon = anon
        self.root = root
        self.schema_nodes = list(schema.graph.nodes)
        
        for node,func in self.funcs.items():
            if isinstance(func,DiagramSchema):
                raise ValueError(f"DiagramSchema nodes are not allowed in a Diagram, they must be instantiated before being added to a Diagram")

    
    def reset(self):
        self.state = None
        self.finished=None
        self.output = None
        self.run_uid = str(uuid.uuid4())

        for node,func in self.funcs.items():
            if _is_attr_method(node_func,'reset'):
                node_func.reset()
    
    @property
    def attrs_to_serialize(self):
        return ['output','finished','next_node','run_uid']

    def __getstate__(self):
        """Dump the state of the diagram and all its nodes into a json serializable dictionary
        """
        state_dict = dict()
        if self.state is not None:
            state_dict['state'] = self.state.model_dump()
        else:
            state_dict['state'] = None

        for attr in self.attrs_to_serialize:
            if hasattr(self,attr):
                state_dict[attr] = getattr(self,attr)

        if not self.anon:
            state_dict['node_state'] = dict()
            for node,func in self.funcs.items():
                if _is_attr_method(func,'__getstate__'):
                    state_dict['node_state'][node] = deepcopy(func.__getstate__())
        return state_dict

    def __setstate__(self,state_dict):
        """Load the state of the diagram and all its nodes from a json serializable dictionary
        """
        self.reset()
        if state_dict.get('state',None) is not None:
            self.state = self.state_class(**state_dict['state'])

        for attr in self.attrs_to_serialize:
            setattr(self,attr,state_dict[attr])

        if self.anon:
            return
        
        for node,state_obj in state_dict['node_state'].items():
            if node in self.factored_graph.nodes:
                node_func = self.factored_graph.nodes[node].get('func',None)
                if _is_attr_method(node_func,'__setstate__'):
                    node_func_copy = deepcopy(node_func)
                    logger.debug(f"Creating copy of {node_func}, {node_func_copy} for loading state")
                    node_func_copy.__setstate__(deepcopy(state_obj))
                    self.factored_graph.nodes[node]['func'] = node_func_copy
                else:
                    raise ValueError(f"Node '{node}' state {state_obj} was saved but has no __setstate__ method is found for {node_func}")


In [ ]:
# TODO from here, make simple diagrams decision and flow, with state and without,
# and compound diagrams
# and nested subdiagrams

# TODO, put diagram basics in the schema call it diagram_base

# this nb will be called execution and be put after the declerative definition nb
# and excution will patch the diagram from the schema logic

# in init we will import all of these modules and therefore patch the classes
# that way we can use the declerative syntax to test the execution logic


## Accessing state and functions

In [ ]:
#| export
@patch
def get_root(self:Diagram):
    if self.anon:
        return self.root
    else:
        return self

@patch
def __getitem__(self:Diagram,key):
    return self.get_root().funcs.get(key,None)  

@patch
def __setitem__(self:Diagram,key,value):
    self.get_root().funcs[key] = value


@patch
def get_state(self:Diagram,key):
    state = self.get_root().state
    return get_state_key(state,key)

@patch
def set_state(self:Diagram,key,value):
    state = self.get_root().state
    set_state_key(state,key,value)


def simplify_output(input_):
    if isinstance(input_,dict) and len(input_) ==1 and 0 in input_:
        output = input_[0]
    else:
        output = input_ 
    return output

In [ ]:
#| export

@patch
def _set_node_output(self:Diagram, node, output):
    # for each write state, get the output at the port position and set it to the state at based on the key
    graph = self.graph
    for key,mapping in graph.nodes[node].get('write_state',{}).items():      
        object_to_write = simplify_output(map_object(output,mapping))
        logger.debug(f"Setting state '{key}' based on output {output}\n"
                    f"after mapping it to {object_to_write}\n"
                    f"with mapping {mapping}")
        self.set_state(key,object_to_write)



In [ ]:
#|export

@patch
def _get_state_input_keys(self:Diagram,node):
    return list(self.graph.nodes[node].get('read_state',{}).keys())
@patch
def _get_state_input(self:Diagram,node):
    keys = self._get_state_input_keys(node)
    return {('state',key):simplify_output(self.get_state(key)) for key in keys}

@patch
def compute_node_input(self:Diagram,node,previous_outputs,state,raw_input=False,partial=False):
    """Prepare the input for the next node execution"""
    
    graph = self.factored_graph
    logger.debug(f"diagram {self.name} preparing input for node '{node}' based on previous outputs {previous_outputs}")

    port_mappings = graph.nodes[node]['mapping']

    state_objects = self._get_state_input(node)

    if raw_input:
        object_to_map = state_objects
    else:
        object_to_map = previous_outputs | state_objects

    func_input = multi_map(object_to_map,port_mappings)

    if raw_input:
        if isinstance(previous_outputs,dict):
            func_input = previous_outputs|func_input
        else:
            func_input = {0:simplify_output(previous_outputs)}|func_input

    logger.debug(f"diagram {self.name} for node '{node}' input prepared:{func_input}")

    return func_input


In [ ]:
#| export
@patch

def _before_node(self:Diagram,node,input_):
    # if we got a validator, validate the input
    self._validate_node_input(node,input_)
    
    return self[node]

@patch
def _after_node(self:Diagram,node,output):
    self._validate_node_output(node,output)
    # set the output to the outbound state
    self._set_node_output(node,output)
    return 

@patch
def _validate_node_input(self:Diagram,node,func_input):
    # get the input validator
    # if validator is basemodel, and there is only a singel arg, validate it
    # if validator is basemodel, and there are only kwargs, validate them as a dict
    # if validator is basemodel, and there are args and kwargs, log a warning

    # if validator is callable, run it on input as args and kwargs
    return 

@patch
def _validate_node_output(self:Diagram,node,output):
    # get the output validator
    # if validator is basemodel validate it against it
    # if validator is callable, run it on output as a single arg
    return 

## Traces

In [ ]:
#| export
from datetime import datetime
from typing import Optional
from pydantic import BaseModel, computed_field
from pprint import pprint
from typing import Dict,Any,Literal,Union,List

from pprint import pformat
from textwrap import indent

from pprint import PrettyPrinter
import reprlib


In [ ]:
#| export
class TruncatedPrettyPrinter(PrettyPrinter):
    def __init__(self, *args, str_length=100, **kwargs):
        super().__init__(*args, **kwargs)
        self.str_length = str_length

    def _format(self, obj, stream, indent, allowance, context, level):
        if isinstance(obj, str):
            if len(obj) > self.str_length:
                # Truncate string and add ...
                stream.write(repr(obj[:self.str_length] + '...'))
                return
        super()._format(obj, stream, indent, allowance, context, level)

In [ ]:
#| export
class Trace(BaseModel):
    model_config = {"arbitrary_types_allowed": True}
    run_uid: str
    node_uid: str
    node_name: List[str]
    input_idx: List[Union[int,None]] = [None]
    node_func: Optional[Any] = None
    input_state_keys: Optional[set[str]] = None
    input_: Any
    output: Optional[Any] = None
    run_type: DiagramType
    start_time: Optional[datetime] = None
    end_time: Optional[datetime] = None

    @computed_field
    def duration(self) -> Optional[datetime]:
        """Return the duration of the node execution in seconds"""
        if self.start_time is None or self.end_time is None:
            return None
        return (self.end_time - self.start_time).total_seconds()

    def nest(self,name,idx):
        self.node_name.insert(0,name)
        self.input_idx.insert(0,idx)

    def pretty_name(self):
        name_parts = []
        for name,idx in zip(self.node_name,self.input_idx):
            if idx is None:
                name_parts.append(name)
            else:
                name_parts.append(f"{name}[{idx}]")
        return '.'.join(name_parts)

    def pformat(self,show_input=True,show_output=True,show_input_state=True,depth=None,indent=2,width=80,str_max_length=None):

        data = {}
        
        if show_input_state == False and isinstance(printable_input,dict):
            printable_input = {k:v for k,v in self.input_.items() if k not in self.input_state_keys}
        else:
            printable_input = self.input_
    
        if show_input:
            data['input'] = printable_input
        if show_output:
            data['output'] = self.output

        if str_max_length is not None:
            pp = TruncatedPrettyPrinter(str_length=str_max_length,indent=indent,width=width,depth=depth)
        else:
            pp = PrettyPrinter(indent=indent,width=width,depth=depth)
        return pp.pformat(data)

    def write(self,string,file=None):
        if file is None:
            fptr = sys.stdout
        else:
            fptr = open(file,'a')
        fptr.write(string+'\n')
        if file is not None:
            fptr.close()

    def pprint(self,
        show_input=True,
        show_output=True,
        show_input_state=True,
        depth=None,
        indent=2,
        width=80,
        str_max_length=None,
        skip_passthrough=False,
        file=None):
        """
        Print the trace in a pretty format
        Args:
            show_input (bool, default=True): Whether to show the input data
            show_output (bool, default=True): Whether to show the output data 
            show_input_state (bool, default=True): Whether to show input state data
            depth (int, optional): Maximum depth to print nested structures
            indent (int, default=2): Number of spaces for each indentation level
            width (int, default=80): Maximum line width for pretty printing
            str_max_length (int, optional): Maximum length for string values before truncating
            skip_passthrough (bool, default=False): Whether to skip printing passthrough nodes (nodes with no function)
            file (str, optional): File to write the output to. Defaults to None. which prints to stdout
        """
        if skip_passthrough and self.node_func is None:
            return
        self.write(f'Node {self.pretty_name()}:',file=file)
        self.write(self.pformat(show_input,show_output,show_input_state,depth,indent,width,str_max_length),file=file)
        self.write('='*width,file=file)


In [ ]:
#| export
@patch
def prep_trace(self:Diagram,node,input_,output,type=DiagramType.decision,idx=None,start_time=None,end_time=None):
    input_state_keys = self.graph.nodes[node].get('read_state',{}).keys()
    return Trace(
        run_uid = self.run_uid,
        node_uid = str(uuid.uuid4()),
        node_name = [node],
        input_idx = [idx],
        node_func = self[node],
        input_state_keys = input_state_keys,
        input_ = input_,
        output = output,
        run_type = type,
        start_time = start_time,
        end_time = end_time
        )


## Running nodes

In [ ]:
#| export

@patch
async def run_node(self:Diagram,node,input_,idx=None):

    graph = self.graph
    
    func = self._before_node(node,input_)
    
    logger.debug(f"Running node '{node}[{idx}]' with input {input_} and state {self.state}")
    if func is None:
        output = simplify_output(input_)
    else:
        args,kwargs = object_to_args_kwargs(input_)
        try:
            # run the node
            output = await maybe_await(func,args,kwargs)
        except Exception as e:
            raise ValueError(f"When running node '{node}[{idx}]':\n"
                f"Function {func}(args={args},kwargs={kwargs})\nreturned\nError '{e}'\n") from e

    logger.debug(f"{node}({input_})={output}")
    self._after_node(node,output) 
    return input_,output


@patch
async def run_subdiagram_iter(self:Diagram,node,input_,subdiagram,idx=None):
    """
    Run a subdiagram as a node
    yields traces from the subdiagram
    returns the output of the subdiagram and the subdiagram itself
    """
    graph = self.graph

    _ = self._before_node(node,input_)

    func = subdiagram

    # run the sub diagram via its run method as an iterator,
    # yield traces from the sub diagram
    if self.derive_state:
        state = self.state
    else:
        state = None
    async for trace in subdiagram.arun(input_,state = state,progress_bars=self.progress_bars):
        # append the name of the node to the last node and nodes with a '.' delimiter
        if not self.trace_nested:
            # if we dont want nested trace print yielding them.
            continue
        if not subdiagram.anon:
            trace.nest(node,idx)
        yield trace

    # after the subdiagram is done, set the output to the outbound state
    self._after_node(node,subdiagram.output)
    return




### Run Decision Diagrams


In [ ]:
#| export

decision_logger = logging.getLogger(f'{__name__}.decision')

@patch
async def arun_decision(self:Diagram,input_,state,**kwargs):

    logger.debug(f"Running decision diagram {self.name} with input {input_}")
    if self.finished in [True,None]:
        self.finished = False
        current_node = self.start_node
    else:
        current_node = self.next_node

    graph = self.graph
    # no need to map ports on iteration
    raw_input = True

    while True:
        func = self[current_node]
        is_sub_diagram = isinstance(func,Diagram)
        is_anon_sub_diagram = is_sub_diagram and func.anon

        input_ = self.compute_node_input(current_node,input_,self.state,raw_input,partial=True)

        start_time = datetime.now()
        if is_sub_diagram:
            subdiagram = func
            async for trace in self.run_subdiagram_iter(current_node,input_,subdiagram,**kwargs):
                yield trace
            output = subdiagram.output
        else:
            _,output = await self.run_node(current_node,input_,**kwargs)
        # TODO the delta time should also work for sub diagrams with breakpoints, for the entire duration until they finished, excluding waiting time to continue.
        end_time = datetime.now()
        
        sub_dir_break = is_sub_diagram and subdiagram.finished == False

        raw_input = False
        if not (is_anon_sub_diagram or sub_dir_break):
            yield self.prep_trace(current_node,input_=input_,output=output,type=DiagramType.decision)

        if sub_dir_break:
            next_node = current_node
            logger.debug(f"Sub diagram {current_node} is not finished, will continue from it when run again")
        else:
            next_node = self.choose_next_node(current_node,output)
            logger.debug(f"Choosing next node after {current_node} with output {output}: {next_node}")
        
        is_end = next_node is None

        is_break = sub_dir_break or (not is_end and graph.nodes[next_node].get('is_break',False))

        # if we are at the end or a breakpoint,
        # prep for the next run and exit
        if is_end:
            self.finished = True
        if is_end or is_break:
            # set the output as the output of the last node we run
            saved_output = output
            
            self.output = simplify_output(output)
            # set the next node to run
            self.next_node = next_node
            return 
        
        input_ = {current_node:output}
        current_node = next_node

    logger.debug(f"Decision diagram {self.name} finished with output {self.output}")
    # should never get here
    return 


In [ ]:
#| export
@patch
def choose_next_node(self:Diagram,node,output):
    """
    Based on the output of the last node, choose the next node
    returns the next node to run, or None if we are at the end
    """

    graph = self.graph
    if node ==self.end_node:
        return None

    # get a dict with each condition and its target node 
    # and get the default target node
    # based on the outgoing edges from the node

    out_going_edges = {target:data for source,target,data in graph.out_edges(node,data=True)}
    condition_per_target = {}
    for target,data in out_going_edges.items():
        if 'condition' in data:
            condition_per_target[target] = data['condition']
        else:
            default_target = target
    logger.debug(f"Condition per target for node '{node}': {condition_per_target}")
    # run the condition functions of each target node on the output
    condition_results = {}
    for target,condition_func in condition_per_target.items():
        try:
            condition_result = condition_func(output)
        except Exception as e:
            raise ValueError(f"When choosing next node after {node}:\n"
                f"Condition function {condition_func}({output}) returned \n{type(e).__name__}:'{e}'"
                ).with_traceback(e.__traceback__)

        # if any condition function returned something other than a boolean, raise an error
        if not isinstance(condition_result,bool):
            raise ValueError(f"When choosing next node after {node}:\n"
                f"Condition function {condition_func} returned {condition_result} which is not a boolean when evaluated on output {output}")
        condition_results[target] = condition_result

    logger.debug(f"Condition results for node '{node}' with output {output}: {condition_results}")
    # if there are more than 2 trues
        # raise an error
    num_trues = sum(condition_results.values())
    if num_trues > 1:
        raise ValueError(f"When choosing next node after {node}:\n"
            f"More than one condition function succeeded when evaluated on output {output}"
            f"condition results: {condition_results}"
            )
    # if there is one true, return the target node of that condition
    elif num_trues == 1:
        chosen_target = next(target for target,result in condition_results.items() if result)
    # if there are no trues, return the default target
    else:
        chosen_target = default_target
    
    return chosen_target



### Run Flow Diagrams


In [ ]:
#| export
from stringdale.core import (
    new_combinations,
    merge_list_dicts,
    dict_cartesian_product
    )

In [ ]:
#| export

flow_logger = logging.getLogger(f'{__name__}.flow')


async def _wait_for_tasks( tasks):
    """Wait for any task to complete from any of the task dictionaries"""
    all_tasks = (*tasks['regular'],*tasks['iterator'],*tasks['subdiagram'])
    if not all_tasks:
        return None
    done, _ = await asyncio.wait(
        all_tasks,
        return_when=asyncio.FIRST_COMPLETED
    )
    return done


In [ ]:
#| export
import enum

In [ ]:
#| export
def _has_foreach(graph,node):
    return graph.nodes[node].get('for_each',list()) != []

class NodeState(enum.Enum):
    waiting = 0
    running = 1
    finished = 2

@patch
def _did_all_fathers_run(self:Diagram,graph,node):
    fathers = list(graph.predecessors(node))
    flow_logger.debug(f"Checking if all fathers {fathers} for node '{node}' have run at least once. node state: {self.node_state}")
    return all(self.node_state[father] != NodeState.waiting for father in fathers)

@patch
def _did_all_fathers_finish(self:Diagram,graph,node):
    fathers = list(graph.predecessors(node))
    flow_logger.debug(f"Checking if all fathers {fathers} for node '{node}' have finished. node state: {self.node_state}")
    return all(self.node_state[father] == NodeState.finished for father in fathers)

@patch
def _running_node_tasks(self:Diagram,node):
    graph = self.graph
    task_nodes = set()
    for tasks_per_type in self.tasks.values():
        for task in tasks_per_type.values():
            task_nodes.add(task[0])
    return task_nodes


@patch
def can_generate_new_input(self:Diagram,graph,node):
    flow_logger.debug(f"Checking if we can generate new input for node '{node}'. node state: {self.node_state}")
    if _has_foreach(graph,node):
        flow_logger.debug(f"Node '{node}' has foreach, checking if all fathers have run at least once")
        res =  self._did_all_fathers_run(graph,node)
    else:
        flow_logger.debug(f"Node '{node}' does not have foreach, checking if all fathers have finished")
        res = self._did_all_fathers_finish(graph,node)

    flow_logger.debug(f"Node '{node}' can generate new input: {res}")
    return res

@patch
def update_state(self:Diagram,graph,node):
    if self._did_all_fathers_finish(graph,node) and node not in self._running_node_tasks(graph):
        new_state = NodeState.finished
    else:
        new_state = NodeState.running
    flow_logger.debug(f"Updating state for node '{node}'. new state: {new_state}")
    self.node_state[node] = new_state


    # self is running is any of its fathers are still running and it is not a foreach node
    # self is finished if all fathers are finished and it is not a 


In [ ]:
#| export
@patch
def enqueue_task(self:Diagram, node, input_, idx):
    """
    Create appropriate task type and add to corresponding task dict.
    Handles AsyncIter initialization and tracks task creation time.
    
    Args:
        node: Node to execute
        input_: Input data for the node
        idx: Execution index for the node
        tasks: Dict of regular tasks
        iterator_tasks: Dict of iterator tasks
        subdiagram_tasks: Dict of subdiagram tasks
    """
    tasks = self.tasks
    func = self[node]
    creation_time = datetime.now()
    
    if isinstance(func, Diagram):
        # Create subdiagram task
        # copy the subdiagram so that we dont share state between them if they are used multiple times or in foreach
        subdiagram = copy(func)
        subdiagram_iter = self.run_subdiagram_iter(node, input_,subdiagram,idx)
        task = asyncio.create_task(subdiagram_iter.__anext__())
        task_info = (node, idx, subdiagram, subdiagram_iter, input_, creation_time)
        tasks['subdiagram'][task] = task_info
        

    else:
        # Create regular task
        task = asyncio.create_task(self.run_node(node, input_,idx))
        task_info = (node, idx, func, None ,input_, creation_time)
        tasks['regular'][task] = task_info

    if isinstance(input_,dict):
        pretty_input_ = input_
    elif isinstance(input_,FunctionInput):
        pretty_input_ = input_.to_positional_kwargs()
    else:
        pretty_input_ = input_
    input_str = ', '.join([f"{k}={v}" for k,v in pretty_input_.items()])
    flow_logger.debug(f"Enqueued {task.get_name()} for {node}[{idx}]({input_str})")
        

In [ ]:
#| export

def _pretty_tasks(tasks):
    pretty_tasks = {}
    for task_type in tasks:
        pretty_tasks[task_type] = {task.get_name():f'{node}[{idx}]({input_})' for task,(node,idx,_,input_,_) in tasks[task_type].items()}
    return pretty_tasks

@patch
def handle_finished_task(self:Diagram, task):
    """Handle a completed task and return node, trace, and outputs
    
    Args:
        task: The completed asyncio task
        
    Returns:
        Tuple of:
        - node: The node that completed
        - trace: The execution trace (or None)
        - outputs: List of outputs (or None)
    """
    end_time = datetime.now()
    trace = None
    
    tasks = self.tasks
    regular_tasks = tasks['regular']
    iterator_tasks = tasks['iterator']
    subdiagram_tasks = tasks['subdiagram']

    process_output = False

    # Handle regular task
    if task in regular_tasks:
        node, idx, func, _ , input_, start_time = regular_tasks.pop(task)
        input_,output = task.result()
        process_output = True

        trace = self.prep_trace(node, input_=input_, output=output, 
                              type=DiagramType.flow, idx=idx,
                              start_time=start_time, end_time=end_time)

        self.update_state(self.factored_graph,node)
            
    # Handle subdiagram task
    elif task in subdiagram_tasks:
        node, idx, subdiagram, subdiagram_iter, input_, start_time = subdiagram_tasks.pop(task)
        try:
            trace = task.result() # Already a trace
            # Queue up next subdiagram trace
            next_task = asyncio.create_task(subdiagram_iter.__anext__())
            subdiagram_tasks[next_task] = (node, idx,subdiagram, subdiagram_iter, 
                                         input_, datetime.now())
            
        except StopAsyncIteration:
            # Subdiagram complete - prep final trace and get output
            output = subdiagram.output
            process_output = True
            
            if not subdiagram.anon:
                trace = self.prep_trace(node, input_=input_, output=output,
                                  type=DiagramType.flow, idx=idx,
                                  start_time=start_time, end_time=end_time)

        
            self.update_state(self.factored_graph,node)
                
    # Apply node output transformations

    if process_output:
        node_data = self.factored_graph.nodes[node]

        for_each = node_data.get('for_each',None)
        if for_each is not None:
            output = [output]
            if node_data.get('flat', False):
                output = [item for output in output for item in output]
            if node_data.get('filter', False):
                output = [output for output in output if output]
    else:
        output = None
    
    return node, trace, output

In [ ]:
#| export

@wrap_exception("""When trying to map previous outputs {{prev_outputs}}
to inputs for node {{node}} with port mapping {{port_mapping}}\n
got partial input {{partial_input}}
regular inputs {{regular_inputs}}
unused stream inputs {{unused_stream_inputs}}
""")
def _map_to_inputs(node,prev_outputs,port_mapping,is_father_stream,for_each_keys,regular_inputs,unused_stream_inputs):

    

    flow_logger.debug(f"Mapping previous outputs to inputs for node '{node}':\n"
        f"prev_outputs: {prev_outputs}\n"
        f"port_mapping: {port_mapping}\n"
        f"for_each_keys: {for_each_keys}"
        )

    if is_father_stream:
        list_keys = list(prev_outputs.keys())
    else:
        list_keys = []
    partial_input = multi_map(prev_outputs,port_mapping,as_list_keys=list_keys)

    for key,value in partial_input.items():
        if key in for_each_keys:
            assign_to = unused_stream_inputs
        else:
            assign_to = regular_inputs
        if is_father_stream:
            if not key in assign_to:
                assign_to[key] = []
            assign_to[key].extend(value)
        else:
            assign_to[key] = value
    flow_logger.debug(f"\n"
            f"partial_input: {partial_input}\n"
            f"regular_inputs: {regular_inputs}\n"
            f"unused_stream_inputs: {unused_stream_inputs}\n"
            )



@patch
def enqueue_new_input(self:Diagram,graph,node,father_node,new_father_outputs):

    """
    Lets look at this example
    U.(x,y)->V.(a,b)  V foreach a  U is a stream node
    W.(z,w)->V.(c,d)  V foreach d  W is a regular node
    M.(a,b)->V.(e,f)  V for each e,f, M is a stream node
    N.(a,b)->V.(g,h)  V for each g,h, N is a regular node

    We need to have all 8 of a,b,c,d,e,f,g,h
    Since W is regular, once we compute it, we portmap it to c,d and overide them
    Since U is a stream node, we need to get every item from it and extend the port mappings
    Both W and U have a regular port, so we need to wait for both to finish
    N is a regular node but all it's outputs are foreach nodes, 
        so we dont need to wait for it but we cant run until it finishes

    M only has foreach nodes, so we dont need to wait for it
    We need to wait till for both ot finish since both have a foreach 


    then we have a mapped dict of:
    {
     a: a list since U is a stream node
     b: a list since U is a stream node
     c: some object since W is a regular node
     d: a list since we need to for each it (we need to check this)
     e-h: lists since we are foreach
    }
    """
    inputs_per_node = self.inputs_per_node
    is_father_stream = _has_foreach(graph,father_node)
    used_stream_inputs = inputs_per_node[node]['stream_used']
    unused_stream_inputs = inputs_per_node[node]['stream_unused']
    regular_inputs = inputs_per_node[node]['regular']
    
    graph = self.factored_graph
    port_mapping = graph.nodes[node]['mapping']
    for_each_keys = graph.nodes[node].get('for_each',list())

    flow_logger.debug(f"Trying to enqueue new input for node '{node}' with father node '{father_node}' and new father outputs {new_father_outputs}")

    # map the new father outputs to inputs
    _map_to_inputs(node,new_father_outputs,port_mapping,is_father_stream,for_each_keys,regular_inputs,unused_stream_inputs)

    if not self.can_generate_new_input(graph,node):
        flow_logger.debug(f"Not all father nodes have finished for node '{node}', skipping enqueueing new input")
        return

    # load state first time we need it
    state_keys = self._get_state_input_keys(node)
    state_target_ports = list(set(itertools.chain(
        *[list(v.keys()) for k,v in graph.nodes[node].get('read_state',{}).items()]
        )))
    if len(state_keys) == 0:
        need_to_load_state = False
    else:
        need_to_load_state = False
        for state_target_port in state_target_ports:
            if state_target_port not in regular_inputs and state_target_port not in used_stream_inputs:
                need_to_load_state = True
                break

    
    if need_to_load_state:
        flow_logger.debug(f"Need to load state keys {state_keys} for node '{node}', mapping state")
        state_values = self._get_state_input(node)
        _map_to_inputs(node,state_values,port_mapping,False,for_each_keys,regular_inputs,unused_stream_inputs)


    # merge foreach inputs to generate new tasks
    new_input_batches = []
    flow_logger.debug(f"Merging input streams to generate new tasks for node '{node}':\n"
                f"used_stream_inputs: {used_stream_inputs}\n"
                f"unused_stream_inputs: {unused_stream_inputs}\n"
                f"regular_inputs: {regular_inputs}\n"
                )

    if len(unused_stream_inputs) > 0:
        for merged_stream_input in new_combinations(unused_stream_inputs,used_stream_inputs):
            input_ = merged_stream_input|regular_inputs
            new_input_batches.append(input_)
    else:
        new_input_batches = [regular_inputs]

    flow_logger.debug(f"new_input_batches: {new_input_batches}, splitting by foreach keys: {for_each_keys}")
    if len(for_each_keys) > 0:
        new_inputs = []
        for batch in new_input_batches:
            try:
                new_inputs.extend(dict_cartesian_product(batch,for_each_keys))
            except KeyError as e:
                raise e
    else:
        new_inputs = new_input_batches

    new_inputs = [dict(input_) for input_ in new_inputs]
    # enqueue the new inputs to task
    # if len(new_inputs) == 0:
    #     raise AssertionError(f'No new inputs were generated even though all father nodes have finished for {node}')

    flow_logger.debug(f"Enqueuing {len(new_inputs)} new inputs for node '{node}':\n"
                f"new_inputs: {new_inputs}\n")
    for input_ in new_inputs:
        self.enqueue_task(node,input_,idx=self._get_next_index(node,self.counters))

    # now we move all the unused stream inputs to the used stream inputs
    for key in unused_stream_inputs:
        if not key in used_stream_inputs:
            used_stream_inputs[key] = []
        used_stream_inputs[key].extend(unused_stream_inputs[key])
        unused_stream_inputs[key] = []

    return 


In [ ]:
#| export

@patch
def _get_next_index(self:Diagram,node,counters):
    if _has_foreach(self.factored_graph,node):
        return next(counters[node])
    else:
        return None

@patch
async def arun_flow(self:Diagram, input_, state=None):
    """Run a flow diagram asynchronously, yielding traces for each node execution"""
    graph = self.graph
    self.counters = {node: itertools.count() for node in graph.nodes}
    
    # Track three types of tasks
    tasks = {
        'regular':{},
        'iterator':{},
        'subdiagram':{}
    }
    self.tasks = tasks
    
    inputs_per_node = {}
    for node in graph.nodes:
        inputs_per_node[node] = {
            'regular':{},
            'stream_used':{},
            'stream_unused':{},
            }

    self.inputs_per_node = inputs_per_node

    self.node_state = {node:NodeState.waiting for node in graph.nodes}

    flow_logger.debug(f"Running flow diagram {self.name} with input {input_}")
    # enqueue the start node
    # Start with initial task
    
    # TODO add support for progress bars
    # after wait for task, update the progress bar based on node
    # in enqueue_new_input, update the progress bar total based on the number of new inputs and reset 
    input_ = self.compute_node_input(self.start_node,input_,state,raw_input=True)

    self.enqueue_task(self.start_node,input_, idx=self._get_next_index(self.start_node,self.counters))
    
    # Main processing loop
    while tasks or iterator_tasks or subdiagram_tasks:
        done_tasks = await _wait_for_tasks(tasks)
        
        if not done_tasks:
            break
        
        flow_logger.debug(f"Done tasks: {[task.get_name() for task in done_tasks]}")
        for task in done_tasks:
            node,trace,outputs = self.handle_finished_task(task)
            if trace is not None:
                yield trace
            if outputs is not None:
                for successor in graph.successors(node):
                    logger.debug(f"Enqueuing new input for successor {successor} of node {node}")
                    self.enqueue_new_input(graph,successor,node,{node:outputs})
            if node == self.end_node:
                self.finished = True
                self.output = outputs
                break

    self.node_state = None
    self.tasks = None
    self.inputs_per_node = None
    flow_logger.debug(f"Flow diagram {self.name} finished with output {self.output}")

### Main execution functions

In [ ]:
#| export

@wrap_exception("""Error when loading state {{outside_state}} with current state {{current_state}}""")
@patch
def load_external_state(self:Diagram,outside_state=None):
    # TODO maybe simply override all keys of the current state with the outside state
    current_state = self.state
    if outside_state is None:
        if current_state is None:
            return self.state_class()
        else:
            return current_state

    if isinstance(outside_state,BaseModel):
        outside_state_dict = outside_state.model_dump()
    else:
        outside_state_dict = outside_state

    if current_state is None:
        current_state_dict = {}
    else:
        current_state_dict = current_state.model_dump()

    new_state = self.state_class(**(outside_state_dict| current_state_dict))
    
    return new_state
    

In [ ]:
#| export

@patch
async def arun(self:Diagram, input:Any ,state:Union[BaseModel,Dict]=None,progress_bars:bool=True,trace_nested:bool=True):
    """
    Asynchronously run the diagram with the given input and state.

    Args:
        input: The input data to process through the diagram
        state: Optional state to initialize the diagram with
        progress_bars: Whether to display progress bars during execution (default True). Deprecated.
        trace_nested: Whether to trace nested diagram execution (default True)

    Yields:
        Trace objects containing execution state at each step
    """
    
    if self.finished in [True,None]:   
        # meaning we have to run the diagram from scratch
        self.reset()

    self.state = self.load_external_state(state)

    self.trace_nested = trace_nested

    self.progress_bars = progress_bars
    self.input = input

    try:
        if self.type == DiagramType.decision:
            async for trace in self.arun_decision(input,state):
                yield trace
        elif self.type == DiagramType.flow:
            async for trace in self.arun_flow(input,state):
                yield trace
        else:
            raise ValueError(f"Diagram type {self.type} is not supported")
    except Exception as e:
        raise e from None    

@patch
def run(self:Diagram, input:Any ,state:Union[BaseModel,Dict]=None,progress_bars:bool=True,trace_nested:bool=True):
    """
    Run the diagram with the given input and state.

    Args:
        input: The input data to process through the diagram
        state: Optional state to initialize the diagram with
        progress_bars: Whether to display progress bars during execution (default True). Deprecated.
        trace_nested: Whether to trace nested diagram execution (default True)

    Yields:
        Trace objects containing execution state at each step
    """
    async_gen = self.arun(input,state,progress_bars,trace_nested)
    loop = asyncio.get_event_loop()
    loop.set_task_factory(asyncio.eager_task_factory)
    try:
        while True:
            yield loop.run_until_complete(async_gen.__anext__())
    except StopAsyncIteration:
        logger.debug('StopAsyncIteration recieved, ending run')
    except Exception as e:
        raise e from None

@patch
def run_all(self:Diagram, input:Any ,state:Union[BaseModel,Dict]=None,progress_bars:bool=True,trace_nested:bool=True):
    """
    Run the diagram to completion and return the final output.

    Args:
        input: The input data to process through the diagram
        state: Optional state to initialize the diagram with
        progress_bars: Whether to display progress bars during execution (default True). Deprecated.
        trace_nested: Whether to trace nested diagram execution (default True)

    Returns:
        The final output after diagram execution completes
    """
    for trace in self.run(input,state,progress_bars,trace_nested):
        pass
    return self.output




In [ ]:
# TODO future
# TODO add support for progress bars.

## Export

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()